In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
from data_loader import DataLoader
import numpy as np
import os



## sigin
import signal
import sys

Using TensorFlow backend.


In [2]:
## source from keras-gan

class colorizeSketch():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'anime'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Objectives
        # + Adversarial: Fool domain discriminators
        # + Translation: Minimize MAE between e.g. fake B and true B
        # + Cycle-consistency: Minimize MAE between reconstructed images and original
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, normalize=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalize:
                d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, normalize=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50,load=False):

        start_time = datetime.datetime.now()


        if load == True:
            1+1
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        self.data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))


                # If at save interval => save generated image samples
                if batch_i % 4 == 0 and epoch % 10 == 0  :
                    self.sample_images(epoch, batch_i)
                if epoch % 4 == 0  and batch_i == 10 :
                    self.g_AB.save('./models/model.h5')
                    self.g_AB.save_weights('./models/weights.h5')
                    self.g_BA.save('./models/model2.h5')
                    self.g_BA.save_weights('./models/weights2.h5')

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

def signal_handler(self,sig, frame):
        print('Graceful Interupt')


In [3]:

if __name__ == '__main__':


    gan = colorizeSketch()
    
    gan.train(epochs=1000, batch_size=5, sample_interval=100,load=False)


C:\Users\me\Anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/167] time: 0:01:03.986467, [d_loss: 18.333263, g_loss: 107.123444]
[0] [1/167] time: 0:01:06.141620, [d_loss: 42.335957, g_loss: 215.186401]
[0] [2/167] time: 0:01:07.720681, [d_loss: 30.053833, g_loss: 14.988588]
[0] [3/167] time: 0:01:09.311599, [d_loss: 10.428560, g_loss: 5.325154]
[0] [4/167] time: 0:01:10.904636, [d_loss: 4.762024, g_loss: 6.379376]
[0] [5/167] time: 0:01:12.847308, [d_loss: 1.666870, g_loss: 4.963026]
[0] [6/167] time: 0:01:14.439600, [d_loss: 0.646876, g_loss: 4.206460]
[0] [7/167] time: 0:01:16.016181, [d_loss: 0.442851, g_loss: 4.112506]
[0] [8/167] time: 0:01:17.605643, [d_loss: 0.287234, g_loss: 3.926510]
[0] [9/167] time: 0:01:19.537252, [d_loss: 0.314140, g_loss: 4.076351]
[0] [10/167] time: 0:01:21.135214, [d_loss: 0.288650, g_loss: 3.688071]
[0] [11/167] time: 0:01:33.441940, [d_loss: 0.316299, g_loss: 4.403777]
[0] [12/167] time: 0:01:35.030574, [d_loss: 0.250271, g_loss: 4.199466]
[0] [13/167] time: 0:01:36.967319, [d_loss: 0.242417, g_loss: 5.0

[0] [114/167] time: 0:04:27.519183, [d_loss: 0.397732, g_loss: 2.363652]
[0] [115/167] time: 0:04:29.090369, [d_loss: 0.443808, g_loss: 2.755819]
[0] [116/167] time: 0:04:30.697720, [d_loss: 0.462231, g_loss: 2.123581]
[0] [117/167] time: 0:04:32.632442, [d_loss: 0.375952, g_loss: 2.283189]
[0] [118/167] time: 0:04:34.217555, [d_loss: 0.445484, g_loss: 1.981043]
[0] [119/167] time: 0:04:35.805307, [d_loss: 0.446070, g_loss: 2.185266]
[0] [120/167] time: 0:04:37.429783, [d_loss: 0.341625, g_loss: 2.209400]
[0] [121/167] time: 0:04:39.399527, [d_loss: 0.476495, g_loss: 3.259068]
[0] [122/167] time: 0:04:41.020756, [d_loss: 0.642622, g_loss: 2.069368]
[0] [123/167] time: 0:04:42.645434, [d_loss: 0.522658, g_loss: 2.529253]
[0] [124/167] time: 0:04:44.293154, [d_loss: 0.483209, g_loss: 2.765120]
[0] [125/167] time: 0:04:46.285403, [d_loss: 0.489490, g_loss: 2.061935]
[0] [126/167] time: 0:04:47.870335, [d_loss: 0.431546, g_loss: 1.913358]
[0] [127/167] time: 0:04:49.449791, [d_loss: 0.4586

[1] [62/167] time: 0:07:32.676167, [d_loss: 0.409604, g_loss: 1.457855]
[1] [63/167] time: 0:07:34.275121, [d_loss: 0.392325, g_loss: 1.555893]
[1] [64/167] time: 0:07:35.848839, [d_loss: 0.313715, g_loss: 1.708733]
[1] [65/167] time: 0:07:37.417643, [d_loss: 0.318295, g_loss: 1.672977]
[1] [66/167] time: 0:07:39.044222, [d_loss: 0.329889, g_loss: 1.624690]
[1] [67/167] time: 0:07:40.633137, [d_loss: 0.342326, g_loss: 1.622900]
[1] [68/167] time: 0:07:42.217710, [d_loss: 0.318229, g_loss: 1.907434]
[1] [69/167] time: 0:07:43.802716, [d_loss: 0.403294, g_loss: 1.789717]
[1] [70/167] time: 0:07:45.366389, [d_loss: 0.326516, g_loss: 1.574657]
[1] [71/167] time: 0:07:46.960012, [d_loss: 0.385758, g_loss: 1.871012]
[1] [72/167] time: 0:07:48.529207, [d_loss: 0.322466, g_loss: 1.643514]
[1] [73/167] time: 0:07:50.123816, [d_loss: 0.355700, g_loss: 1.450955]
[1] [74/167] time: 0:07:51.735315, [d_loss: 0.317172, g_loss: 1.526232]
[1] [75/167] time: 0:07:53.299987, [d_loss: 0.347558, g_loss: 1.

[2] [9/167] time: 0:10:30.991963, [d_loss: 0.311198, g_loss: 1.470230]
[2] [10/167] time: 0:10:32.553190, [d_loss: 0.351695, g_loss: 1.446527]
[2] [11/167] time: 0:10:34.126558, [d_loss: 0.304754, g_loss: 1.651890]
[2] [12/167] time: 0:10:35.732452, [d_loss: 0.329954, g_loss: 1.304190]
[2] [13/167] time: 0:10:37.348006, [d_loss: 0.328677, g_loss: 1.459762]
[2] [14/167] time: 0:10:38.916745, [d_loss: 0.295853, g_loss: 1.403226]
[2] [15/167] time: 0:10:40.512047, [d_loss: 0.336496, g_loss: 1.461242]
[2] [16/167] time: 0:10:42.083542, [d_loss: 0.315624, g_loss: 1.436045]
[2] [17/167] time: 0:10:43.654593, [d_loss: 0.315407, g_loss: 1.562961]
[2] [18/167] time: 0:10:45.251473, [d_loss: 0.324873, g_loss: 1.593625]
[2] [19/167] time: 0:10:46.815290, [d_loss: 0.321109, g_loss: 1.532212]
[2] [20/167] time: 0:10:48.386721, [d_loss: 0.320948, g_loss: 1.424097]
[2] [21/167] time: 0:10:49.956492, [d_loss: 0.312905, g_loss: 1.368352]
[2] [22/167] time: 0:10:51.525555, [d_loss: 0.308874, g_loss: 1.4

...

[153] [43/167] time: 11:22:24.207540, [d_loss: 0.178736, g_loss: 1.719840]
[153] [44/167] time: 11:22:25.838628, [d_loss: 0.194639, g_loss: 1.909156]
[153] [45/167] time: 11:22:27.438227, [d_loss: 0.208663, g_loss: 1.904173]
[153] [46/167] time: 11:22:29.082433, [d_loss: 0.366778, g_loss: 1.306297]
[153] [47/167] time: 11:22:30.676503, [d_loss: 0.175429, g_loss: 1.636318]
[153] [48/167] time: 11:22:32.332491, [d_loss: 0.182192, g_loss: 1.539198]
[153] [49/167] time: 11:22:33.934378, [d_loss: 0.204432, g_loss: 1.689594]
[153] [50/167] time: 11:22:35.533869, [d_loss: 0.165589, g_loss: 1.581077]
[153] [51/167] time: 11:22:37.147285, [d_loss: 0.232514, g_loss: 1.418452]
[153] [52/167] time: 11:22:38.742831, [d_loss: 0.248268, g_loss: 1.399781]
[153] [53/167] time: 11:22:40.342114, [d_loss: 0.196658, g_loss: 1.613869]
[153] [54/167] time: 11:22:41.943404, [d_loss: 0.219672, g_loss: 1.650417]
[153] [55/167] time: 11:22:43.576882, [d_loss: 0.243826, g_loss: 1.460441]
[153] [56/167] time: 11:2

[153] [152/167] time: 11:25:20.392801, [d_loss: 0.147470, g_loss: 1.848470]
[153] [153/167] time: 11:25:21.987034, [d_loss: 0.210840, g_loss: 1.420020]
[153] [154/167] time: 11:25:23.598125, [d_loss: 0.145485, g_loss: 1.526395]
[153] [155/167] time: 11:25:25.189081, [d_loss: 0.145070, g_loss: 1.626387]
[153] [156/167] time: 11:25:26.784982, [d_loss: 0.157931, g_loss: 1.632646]
[153] [157/167] time: 11:25:28.362693, [d_loss: 0.156006, g_loss: 1.970016]
[153] [158/167] time: 11:25:29.967784, [d_loss: 0.197262, g_loss: 1.605568]
[153] [159/167] time: 11:25:31.559080, [d_loss: 0.168842, g_loss: 1.606545]
[153] [160/167] time: 11:25:33.181516, [d_loss: 0.148841, g_loss: 1.580562]
[153] [161/167] time: 11:25:34.799861, [d_loss: 0.146022, g_loss: 1.789322]
[153] [162/167] time: 11:25:36.412970, [d_loss: 0.195181, g_loss: 1.650157]
[153] [163/167] time: 11:25:38.005158, [d_loss: 0.188948, g_loss: 1.806996]
[153] [164/167] time: 11:25:39.631875, [d_loss: 0.171203, g_loss: 1.703478]
[153] [165/1

[154] [96/167] time: 11:28:16.934129, [d_loss: 0.184527, g_loss: 1.485786]
[154] [97/167] time: 11:28:18.578762, [d_loss: 0.220579, g_loss: 1.771944]
[154] [98/167] time: 11:28:20.168943, [d_loss: 0.198208, g_loss: 1.499295]
[154] [99/167] time: 11:28:21.784465, [d_loss: 0.157554, g_loss: 1.627114]
[154] [100/167] time: 11:28:23.392492, [d_loss: 0.148859, g_loss: 1.938207]
[154] [101/167] time: 11:28:24.993917, [d_loss: 0.186875, g_loss: 1.795003]
[154] [102/167] time: 11:28:26.576106, [d_loss: 0.151104, g_loss: 1.739397]
[154] [103/167] time: 11:28:28.189738, [d_loss: 0.262366, g_loss: 1.590763]
[154] [104/167] time: 11:28:29.763421, [d_loss: 0.160069, g_loss: 1.700449]
[154] [105/167] time: 11:28:31.368169, [d_loss: 0.164197, g_loss: 1.687593]
[154] [106/167] time: 11:28:32.981317, [d_loss: 0.184467, g_loss: 1.954503]
[154] [107/167] time: 11:28:34.606942, [d_loss: 0.180059, g_loss: 1.380194]
[154] [108/167] time: 11:28:36.181344, [d_loss: 0.183682, g_loss: 1.933315]
[154] [109/167] 

[155] [39/167] time: 11:31:11.606104, [d_loss: 0.168819, g_loss: 2.106264]
[155] [40/167] time: 11:31:13.213185, [d_loss: 0.295719, g_loss: 2.063213]
[155] [41/167] time: 11:31:14.838102, [d_loss: 0.163415, g_loss: 1.594067]
[155] [42/167] time: 11:31:16.422325, [d_loss: 0.216101, g_loss: 2.290170]
[155] [43/167] time: 11:31:18.023828, [d_loss: 0.169949, g_loss: 1.679880]
[155] [44/167] time: 11:31:19.635973, [d_loss: 0.264321, g_loss: 1.692474]
[155] [45/167] time: 11:31:21.252917, [d_loss: 0.147923, g_loss: 1.682765]
[155] [46/167] time: 11:31:22.876369, [d_loss: 0.214458, g_loss: 1.497255]
[155] [47/167] time: 11:31:24.481693, [d_loss: 0.151903, g_loss: 1.834021]
[155] [48/167] time: 11:31:26.069354, [d_loss: 0.156552, g_loss: 1.587120]
[155] [49/167] time: 11:31:27.657296, [d_loss: 0.148807, g_loss: 1.679698]
[155] [50/167] time: 11:31:29.270134, [d_loss: 0.166040, g_loss: 1.762547]
[155] [51/167] time: 11:31:30.881627, [d_loss: 0.141021, g_loss: 1.730703]
[155] [52/167] time: 11:3

KeyboardInterrupt: 